In [1]:
import requests
import json
import time

API_KEY = "456a129b2990178516e5e69f5a8d5a74"
BASE_URL = "https://api.themoviedb.org/3"


In [6]:
headers = {
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json"
}

def get_movie_details(movie_id):
    url = f"{BASE_URL}/movie/{movie_id}?api_key={API_KEY}&language=en-US"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    return None

def get_movie_credits(movie_id):
    url = f"{BASE_URL}/movie/{movie_id}/credits?api_key={API_KEY}"
    response = requests.get(url)
    if response.status_code == 200:
        credits = response.json()
        director = next((crew["name"] for crew in credits["crew"] if crew["job"] == "Director"), "Unknown")
        cast = [actor["name"] for actor in credits["cast"][:5]]
        return director, cast
    return "Unknown", []

def save_to_json(data, filename="movies.json"):
    with open(filename, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4, ensure_ascii=False)

def fetch_and_store_movies(num_movies=100):
    all_movies = []
    page = 1

    while len(all_movies) < num_movies:
        url = f"{BASE_URL}/movie/popular?api_key={API_KEY}&language=en-US&page={page}"
        response = requests.get(url)

        if response.status_code != 200:
            print(f"Failed to fetch page {page}. Status: {response.status_code}")
            break

        movies = response.json()["results"]
        if not movies:
            print("No more movies available.")
            break

        for movie in movies:
            if len(all_movies) >= num_movies:
                break  # Ensure we do not fetch more than required

            movie_id = movie["id"]
            details = get_movie_details(movie_id)
            if not details:
                continue  # Skip if movie details are missing

            director, cast = get_movie_credits(movie_id)

            movie_data = {
                "title": details["title"],
                "description": details["overview"],
                "director": director,
                "cast": cast,
                "poster_url": f"https://image.tmdb.org/t/p/w500{details['poster_path']}" if details["poster_path"] else "N/A"
            }

            all_movies.append(movie_data)

        page += 1
        time.sleep(1)

    save_to_json(all_movies)
    print(f"Saved {len(all_movies)} movies to movies.json")

In [7]:
fetch_and_store_movies(50)

Failed to fetch page 1. Status: 401
Saved 0 movies to movies.json
